# Analysing memory usage

In [ ]:
from components.model import get_income_and_attitude_weights
import sys
from pympler.asizeof import asizeof


incomes, weights = get_income_and_attitude_weights(700, 0.6)

print(sys.getsizeof(incomes),sys.getsizeof(weights))
asizeof(incomes),asizeof(weights) 
# (5728, 41368) 13.03.24
# (2928, 24568) after np.float32


In [ ]:
from data.canada import uncertain_demand_from_income_and_province, get_end_use_agg_heating_share


demand = uncertain_demand_from_income_and_province(incomes, "Ontario")
agg_heat_share = get_end_use_agg_heating_share("Ontario", 2000)
print(sys.getsizeof(demand),sys.getsizeof(agg_heat_share))
asizeof(demand),asizeof(agg_heat_share)
# (5728, 48)

In [ ]:
from components.technologies import merge_heating_techs_with_share
import pandas as pd

heat_tech_shares = merge_heating_techs_with_share(2000, "Ontario")
print(sys.getsizeof(heat_tech_shares))
# heat_tech_shares[:] = np.float16(heat_tech_shares.values)

asizeof(heat_tech_shares.astype("float16"))

In [ ]:
from components.model import TechnologyAdoptionModel

model = TechnologyAdoptionModel(500, "Ontario", ts_step_length="w")
print(sys.getsizeof(model))
asizeof(model) 
# holy smokes! 963243912 13.03.24
# 693435952 after some float32 conversions
#  35754928 after resampling to "w" ~ 19x less memory
#  25546080 with 500 instead of 700 agents

In [ ]:
fuel_demands = model.schedule.agents[0].potential_fuel_demands
asizeof(fuel_demands),asizeof(fuel_demands.astype("int32"))

In [ ]:
from components.probability import beta_with_mode_at
from components.technologies import Technologies

dist_params = dict(zip(Technologies,[0.5]*len(Technologies)))

atts = model.draw_attitudes_from_distribution(beta_with_mode_at, dist_params)
print(sys.getsizeof(atts))
asizeof(atts)
# 74640